## Machine Learning Project Checklist
1. Frame the problem and look at the bigger picture
2. Get the data
3. Explore the data to gain insights
4. Prepare the data to better expose the underlying data patterns to machine learning algorithms
5. Explore many different models and shortlist the best ones
6. Fine-tune yor models and combine them into a greate solution
7. present your solution
8. Launch, monitor and maintain your system


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:

# Importing the data from Kaggle using kagglehub
import kagglehub

# Download and get the path to the UTKFace dataset
path = kagglehub.dataset_download("jangedoo/utkface-new")
print("Path to dataset files:", path)

path = os.path.join(path, "UTKFace")

In [ ]:

# Set target image size and other parameters
IMG_HEIGHT, IMG_WIDTH = 128, 128  # You can change these dimensions as needed
BATCH_SIZE = 64
EPOCHS = 10  # Adjust epochs depending on your dataset and GPU/CPU


In [4]:

# Initialize lists to hold image data and labels
images = []
ages = []
genders = []

# Loop over each file in the dataset folder
for filename in os.listdir(path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # The filename format is usually: age_gender_race_date.jpg
        try:
            parts = filename.split('_')
            age = int(parts[0])
            gender = int(parts[1])  # Typically 0 for male and 1 for female
        except Exception as e:
            print(f"Skipping file {filename} due to error: {e}")
            continue

        # Load image with the specified target size
        img_path = os.path.join(path, filename)
        img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        img = img_to_array(img)
        images.append(img)
        ages.append(age)
        genders.append(gender)

# Convert lists to numpy arrays
images = np.array(images, dtype="float32")
ages = np.array(ages, dtype="float32")
genders = np.array(genders, dtype="float32")

In [5]:

# Normalize the image pixel values to [0, 1]
images /= 255.0

print("Dataset shapes:")
print("Images:", images.shape)
print("Ages:", ages.shape)
print("Genders:", genders.shape)


Dataset shapes:
Images: (23708, 128, 128, 3)
Ages: (23708,)
Genders: (23708,)


In [6]:

# Split the dataset into training and testing sets
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    images, ages, genders, test_size=0.2, random_state=42)



In [7]:

# Build the multi-output model using the Functional API
input_layer = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Shared convolutional base
# Convolutional base with BatchNormalization and additional filters
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Age regression branch
age_branch = layers.Dense(128, activation='relu')(x)
age_branch = layers.Dropout(0.5)(age_branch)
age_output = layers.Dense(1, name='age')(age_branch)

# Gender classification branch
gender_branch = layers.Dense(128, activation='relu')(x)
gender_branch = layers.Dropout(0.5)(gender_branch)
gender_output = layers.Dense(1, activation='sigmoid', name='gender')(gender_branch)

# Create the model with two outputs
model = keras.Model(inputs=input_layer, outputs=[age_output, gender_output])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [8]:

# Compile the model with different loss functions for each output
model.compile(
    optimizer='adam',
    loss={'age': 'mse', 'gender': 'binary_crossentropy'},
    metrics={'age': 'mae', 'gender': 'accuracy'}
)

# Train the model
history = model.fit(
    X_train,
    {'age': y_age_train, 'gender': y_gender_train},
    validation_data=(X_test, {'age': y_age_test, 'gender': y_gender_test}),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)


Epoch 1/10
297/297 [==============================] - 22s 59ms/step - loss: 293.1034 - age_loss: 292.1107 - gender_loss: 0.9926 - age_mae: 12.8117 - gender_accuracy: 0.5842 - val_loss: 277.7584 - val_age_loss: 277.0777 - val_gender_loss: 0.6806 - val_age_mae: 13.0278 - val_gender_accuracy: 0.5325
Epoch 2/10


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/batch_normalization/FusedBatchNormGradV3' defined at (most recent call last):
    File "c:\Users\ziadt\anaconda3\envs\CV_masterclass_env\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\ziadt\anaconda3\envs\CV_masterclass_env\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\ziadt\anaconda3\envs\CV_masterclass_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\ziadt\anaconda3\envs\CV_masterclass_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\ziadt\anaconda3\envs\CV_masterclass_env\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\ziadt\anaconda3\envs\CV_masterclass_env\lib\site-packages\tornado\platform

In [ ]:
# Plot training history for age and gender losses
plt.figure(figsize=(12, 5))

# Age loss
plt.subplot(1, 2, 1)
plt.plot(history.history['age_loss'], label='Train Age Loss')
plt.plot(history.history['val_age_loss'], label='Val Age Loss')
plt.title('Age Regression Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()

# Gender loss
plt.subplot(1, 2, 2)
plt.plot(history.history['gender_loss'], label='Train Gender Loss')
plt.plot(history.history['val_gender_loss'], label='Val Gender Loss')
plt.title('Gender Classification Loss')
plt.xlabel('Epoch')
plt.ylabel('Binary Crossentropy Loss')
plt.legend()

# age accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['age_mae'], label='Train Age MAE')
plt.plot(history.history['val_age_mae'], label='Val Age MAE')
plt.title('Age Regression MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

# gender accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['gender_accuracy'])

plt.tight_layout()
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the directory containing the test images
test_images_dir = "C:/Users/ziadt/Desktop/Projects/syed assignment/SyedAssign/Zezoo/test images"

# List all image files in the directory (adjust extensions as needed)
image_files = [os.path.join(test_images_dir, f) 
               for f in os.listdir(test_images_dir) 
               if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

print(f"Found {len(image_files)} images for prediction.")

# Define grid layout for displaying images
n_cols = 3
n_rows = (len(image_files) + n_cols - 1) // n_cols
plt.figure(figsize=(n_cols * 5, n_rows * 5))

# Loop over each image, predict, and plot the result
for i, image_path in enumerate(image_files):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

    # Make predictions with your model
    predictions = model.predict(img_array)
    predicted_age = predictions[0].flatten()[0]
    predicted_gender = predictions[1].flatten()[0]
    pred_gender_label = "Female" if predicted_gender >= 0.5 else "Male"

    # Plot the image with its predicted age and gender
    plt.subplot(n_rows, n_cols, i + 1)
    plt.imshow(load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH)))
    plt.title(f"Age: {predicted_age:.1f}\nGender: {pred_gender_label}")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:

# Evaluate the model on test data
image_path =  "C:/Users/ziadt/Desktop/Projects/syed assignment/SyedAssign/Zezoo/test images/Screenshot 2025-03-02 170932.jpg"
img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
img_array = img_to_array(img)

# Normalize the image
img_array = img_array / 255.0

# Add batch dimension
img_array = tf.expand_dims(img_array, 0)

# Make predictions on some test samples
predictions = model.predict(img_array)

predicted_ages = predictions[0].flatten()
predicted_genders = predictions[1].flatten()

# Display the prediction for the single test image
plt.figure(figsize=(5, 5))
plt.imshow(img_array[0])
pred_age = predicted_ages[0]
pred_gender = predicted_genders[0]
# For gender, apply a 0.5 threshold
pred_gender_label = "Female" if pred_gender >= 0.5 else "Male"
plt.title(f"Predicted Age: {pred_age:.1f}\nPredicted Gender: {pred_gender_label}")
plt.axis("off")
plt.show()

In [ ]:
# save the model
model.save(filepath="C:/Users/ziadt/Desktop/Projects/syed assignment/Gen-Age-Predections/trained models/model.h5")